In [2]:
from tqdm.notebook import tqdm
from dotenv import load_dotenv

from m3_sentence_transformer.data_sampler import get_sample_docs_with_all_qrels

load_dotenv()


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [3]:
import weaviate
import weaviate.classes as wvc

client = weaviate.connect_to_local()

sample_path = "neuclir_1_mutli_bge_m3_100k.csv"
collection_name = "neuclir_1_mutli_bge_m3_100k"
collbert_collection_name = "neuclir_1_mutli_bge_m3_100k_colbert"

In [ ]:

docs = get_sample_docs_with_all_qrels(sample_path)


len(docs)

In [13]:
#client.collections.delete("neuclir_1_mutli_bge_m3_100k")

In [4]:
try:
    documents = client.collections.create(
        name=collection_name,
        vectorizer_config=[
            wvc.config.Configure.NamedVectors.none(
                name="title_dense",
                vector_index_config=wvc.config.Configure.VectorIndex.hnsw(
                    vector_cache_max_objects=100000,
                    #quantizer=wvc.config.Reconfigure.VectorIndex.Quantizer.pq(training_limit=100000)  # Set the threshold to begin training
                ),    
            ),
            wvc.config.Configure.NamedVectors.none( 
                name="text_dense",
                vector_index_config=wvc.config.Configure.VectorIndex.hnsw(
                    vector_cache_max_objects=100000,
                   # quantizer=wvc.config.Reconfigure.VectorIndex.Quantizer.pq(training_limit=10000)  # Set the threshold to begin training
                ),
            ),
        ],
        properties=[
            wvc.config.Property(
                name="doc_id",
                data_type=wvc.config.DataType.UUID,
            ),
           # wvc.config.Property(
          #      name="title_sparse",
           #     data_type=wvc.config.DataType.BLOB,
           # ),
          #    wvc.config.Property(
          #      name="document_sparse",
          #      data_type=wvc.config.DataType.BLOB,
          #  ),
          #  wvc.config.Property(
          #      name="title_colbert",
          #      data_type=wvc.config.DataType.BLOB,
          #  ),
        #   wvc.config.Property(
         #       name="document_colbert",
         #       data_type=wvc.config.DataType.BLOB,
          #  ),
            wvc.config.Property(
                name="title",
                data_type=wvc.config.DataType.TEXT,
            ),
            wvc.config.Property(
                name="text",
                data_type=wvc.config.DataType.TEXT,
            ),
            wvc.config.Property(
                name="url",
                data_type=wvc.config.DataType.TEXT,
            )
        ])
except Exception as e:
    print(e)


/Users/I551385/Library/Caches/pypoetry/virtualenvs/ie691-ir-project-u22LL72s-py3.12/lib/python3.12/site-packages/weaviate/warnings.py:329: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


In [5]:
try:
    documents = client.collections.create(
        name=collbert_collection_name,
        vectorizer_config=[
            wvc.config.Configure.NamedVectors.none(
                name="title_colbert",
                vector_index_config=wvc.config.Configure.VectorIndex.hnsw(
                    vector_cache_max_objects=50000,
                    #quantizer=wvc.config.Configure.VectorIndex.Quantizer.pq(training_limit=30000)  # Set the threshold to begin training
                ),
        )],
        properties=[
            wvc.config.Property(
                name="doc_id",
                data_type=wvc.config.DataType.UUID,
            ),
        ])
except Exception as e:
    print(e)


In [8]:

zh = client.collections.get(collection_name)
aggregation = zh.aggregate.over_all(total_count=True)
print(aggregation.total_count)

44621
